In [ ]:
import pandas as pd
import os

in_file = 'data/history_weathered.csv'
out_file = 'data/with_sessions.csv'
chunksize = 10**3
timeframe = 300 #seconds
min_session_size = 3

session_no = 0

header = True    
for df in pd.read_csv(in_file,chunksize=chunksize,skiprows=1,dtype={'cnt':int,'weekday':int}):
    # set timestamp
    df['weekday']=df['weekday']+1
    df['timestamp'] = pd.to_datetime("2021/09/"+df['weekday'].astype(str)+'/'+df['time'])
    
    # select required columns
    df = df[['wbuser_id','UQ','cnt','timestamp']]
    
    #select sessions
    # TODO: does chunksize always include given session timeframe?

    users = df[['wbuser_id']].drop_duplicates(['wbuser_id']).to_numpy().ravel()
    
    sessions = pd.DataFrame()
    for u in users:
        requests = df.loc[df['wbuser_id']==u]
        if len(requests)<min_session_size:
            continue
        
        prev = None
        pos = 0
        for count,row in requests.iterrows():
            if pos==0:
                prev = row
                pos = 1
                continue
            
            diff = (row['timestamp']-prev['timestamp']).seconds
            if diff<timeframe:
                if pos==1:
                    prev['session_no'] = session_no
                    sessions = sessions.append(prev)
                    pos=2
                    
                row['session_no'] = session_no
                sessions = sessions.append(row)
            else:
                session_no+=1
        
        session_no+=1
    
        # append file
        sessions.to_csv(out_file,
            header=header, mode='a')
        header = False
    